# Evaluation of lower bound heuristic

This notebook documents the evaluation of the beam search construction heuristic used to obtain a lower bound.

In [1]:
using thesis.Instances
using thesis.LocalSearch
using BenchmarkTools
using DataFrames
using Printf
using Latexify
;

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-16


In [2]:
# selection of 20 benchmark instances
instances = [
    MQCPInstance("brock400_1", 0.999,  27),
    MQCPInstance("brock400_2", 0.999,  29),
    MQCPInstance("brock400_2", 0.8  , 187),
    MQCPInstance("brock400_3", 0.999,  31),
    MQCPInstance("brock800_1", 0.9  ,  43),
    MQCPInstance("brock800_2", 0.999,  24),
    MQCPInstance("brock800_2", 0.8  ,  96),
    MQCPInstance("brock800_3", 0.999,  22),
    MQCPInstance("brock800_3", 0.9  ,  43),
    MQCPInstance("brock800_3", 0.8  ,  94),
    MQCPInstance("C1000.9",    0.999,  70),
    MQCPInstance("C1000.9",    0.95 , 222),
    MQCPInstance("C2000.9",    0.999,  82),
    MQCPInstance("C2000.9",    0.95 , 288),
    MQCPInstance("DSJC1000.5", 0.999, 15),
    MQCPInstance("DSJC1000.5", 0.8, 41),
    MQCPInstance("DSJC500.5", 0.999, 13),
    MQCPInstance("gen400_p0.9_55", 0.999, 55),
    MQCPInstance("gen400_p0.9_65", 0.999, 66),
    MQCPInstance("hamming10-4", 0.95, 88),
]
;

In [3]:
function evaluate(guidance_function::GuidanceFunction, guidance_str::String; β=1, expansion_limit=10)

    df = DataFrame(GraphID=String[], 
                   target_γ=Real[], 
                   guidance_func=String[],  
                   β=Int[], 
                   exp_limit=Int[], 
                   time=String[], 
                   obj=Int[], 
                   best_known=Int[])

    for inst in instances
        graph = inst.graph
        γ = inst.target_γ

        t = @elapsed begin 
            S = lower_bound_heuristic(graph, γ, guidance_function; β, expansion_limit)
        end

        push!(df, (
            inst.graph_id,
            inst.target_γ,
            guidance_str,
            β,
            expansion_limit,
            @sprintf("%.2f", t),
            length(S),
            inst.best_known
        ))
    end

    return df
end

evaluate (generic function with 1 method)

### Greedy Construction

In [4]:
greedy = GreedyCompletionHeuristic()
sum_of_neighbors = SumOfNeighborsHeuristic(10, 0.2)

SumOfNeighborsHeuristic(10, 0.2)

#### $\beta = 1, \mathit{expansion\_limit} = 10$

In [5]:
df_beta_1_exp_10_greedy = evaluate(greedy, "greedy"; β=1, expansion_limit=10)
df_beta_1_exp_10_sum = evaluate(sum_of_neighbors, "sum of neighbors"; β=1, expansion_limit=10)

,GraphID,target_γ,guidance_func,β,exp_limit,time,obj,best_known
,String,Real,String,Int64,Int64,String,Int64,Int64
1,brock400_1,0.999,sum of neighbors,1,10,0.23,17,27
2,brock400_2,0.999,sum of neighbors,1,10,0.01,17,29
3,brock400_2,0.8,sum of neighbors,1,10,0.07,163,187
4,brock400_3,0.999,sum of neighbors,1,10,0.01,16,31
5,brock800_1,0.9,sum of neighbors,1,10,0.04,31,43
6,brock800_2,0.999,sum of neighbors,1,10,0.05,14,24
7,brock800_2,0.8,sum of neighbors,1,10,0.07,78,96
8,brock800_3,0.999,sum of neighbors,1,10,0.04,18,22
9,brock800_3,0.9,sum of neighbors,1,10,0.04,32,43


#### $\beta = 5, \mathit{expansion\_limit} = 10$

In [6]:
df_beta_5_exp_10_greedy = evaluate(greedy, "greedy"; β=5, expansion_limit=10)
df_beta_5_exp_10_sum = evaluate(sum_of_neighbors, "sum of neighbors"; β=5, expansion_limit=10)

,GraphID,target_γ,guidance_func,β,exp_limit,time,obj,best_known
,String,Real,String,Int64,Int64,String,Int64,Int64
1,brock400_1,0.999,sum of neighbors,5,10,0.04,19,27
2,brock400_2,0.999,sum of neighbors,5,10,0.03,19,29
3,brock400_2,0.8,sum of neighbors,5,10,0.33,178,187
4,brock400_3,0.999,sum of neighbors,5,10,0.04,17,31
5,brock800_1,0.9,sum of neighbors,5,10,0.11,35,43
6,brock800_2,0.999,sum of neighbors,5,10,0.07,17,24
7,brock800_2,0.8,sum of neighbors,5,10,0.23,77,96
8,brock800_3,0.999,sum of neighbors,5,10,0.06,16,22
9,brock800_3,0.9,sum of neighbors,5,10,0.11,34,43


In [7]:
# latex_str = latexify(df; env=:table, latex=false)
# open("table.txt", "w") do io
#     write(io, latex_str)
# end

#### $\beta = 5, \mathit{expansion\_limit} = 20$

In [8]:
df_beta_5_exp_20_greedy = evaluate(greedy, "greedy"; β=5, expansion_limit=20)
df_beta_5_exp_20_sum = evaluate(sum_of_neighbors, "sum of neighbors"; β=5, expansion_limit=20)

,GraphID,target_γ,guidance_func,β,exp_limit,time,obj,best_known
,String,Real,String,Int64,Int64,String,Int64,Int64
1,brock400_1,0.999,sum of neighbors,5,20,0.04,20,27
2,brock400_2,0.999,sum of neighbors,5,20,0.04,18,29
3,brock400_2,0.8,sum of neighbors,5,20,0.55,178,187
4,brock400_3,0.999,sum of neighbors,5,20,0.04,21,31
5,brock800_1,0.9,sum of neighbors,5,20,0.16,34,43
6,brock800_2,0.999,sum of neighbors,5,20,0.08,15,24
7,brock800_2,0.8,sum of neighbors,5,20,0.38,81,96
8,brock800_3,0.999,sum of neighbors,5,20,0.08,15,22
9,brock800_3,0.9,sum of neighbors,5,20,0.14,33,43


#### $\beta = 10, \mathit{expansion\_limit} = 20$

In [9]:
df_beta_10_exp_20_greedy = evaluate(greedy, "greedy"; β=10, expansion_limit=20)
df_beta_10_exp_20_sum = evaluate(sum_of_neighbors, "sum of neighbors"; β=10, expansion_limit=20)

,GraphID,target_γ,guidance_func,β,exp_limit,time,obj,best_known
,String,Real,String,Int64,Int64,String,Int64,Int64
1,brock400_1,0.999,sum of neighbors,10,20,0.06,17,27
2,brock400_2,0.999,sum of neighbors,10,20,0.08,18,29
3,brock400_2,0.8,sum of neighbors,10,20,1.04,170,187
4,brock400_3,0.999,sum of neighbors,10,20,0.08,19,31
5,brock800_1,0.9,sum of neighbors,10,20,0.28,32,43
6,brock800_2,0.999,sum of neighbors,10,20,0.13,15,24
7,brock800_2,0.8,sum of neighbors,10,20,0.72,77,96
8,brock800_3,0.999,sum of neighbors,10,20,0.13,16,22
9,brock800_3,0.9,sum of neighbors,10,20,0.28,32,43


#### $\beta = 20, \mathit{expansion\_limit} = 50$

In [11]:
df_beta_20_exp_50_greedy = evaluate(greedy, "greedy"; β=20, expansion_limit=50)
df_beta_20_exp_50_sum = evaluate(sum_of_neighbors, "sum of neighbors"; β=20, expansion_limit=50)

,GraphID,target_γ,guidance_func,β,exp_limit,time,obj,best_known
,String,Real,String,Int64,Int64,String,Int64,Int64
1,brock400_1,0.999,sum of neighbors,20,50,0.26,19,27
2,brock400_2,0.999,sum of neighbors,20,50,0.22,17,29
3,brock400_2,0.8,sum of neighbors,20,50,4.46,166,187
4,brock400_3,0.999,sum of neighbors,20,50,0.21,18,31
5,brock800_1,0.9,sum of neighbors,20,50,1.15,34,43
6,brock800_2,0.999,sum of neighbors,20,50,0.40,15,24
7,brock800_2,0.8,sum of neighbors,20,50,3.00,82,96
8,brock800_3,0.999,sum of neighbors,20,50,0.38,15,22
9,brock800_3,0.9,sum of neighbors,20,50,1.06,32,43


In [12]:
using CSV

df_all = vcat(
    df_beta_1_exp_10_greedy,
    df_beta_1_exp_10_sum,
    df_beta_5_exp_10_greedy,
    df_beta_5_exp_10_sum,
    df_beta_5_exp_20_greedy,
    df_beta_5_exp_20_sum,
    df_beta_10_exp_20_greedy,
    df_beta_10_exp_20_sum,
    df_beta_20_exp_50_greedy,
    df_beta_20_exp_50_sum
)

CSV.write("df_lower_bound_heuristic.csv", df_all)

"df_lower_bound_heuristic.csv"